# Diffusion Language Model from scratch (TinyStories) + terminal-style inference GIF

This Colab notebook trains a **discrete diffusion language model** (a masked-denoising Transformer) **from random initialization** on a **small slice of TinyStories**, then runs **diffusion sampling** and exports a **terminal-looking inference GIF** similar to the one you shared.

It’s designed to be:

- **Educational**: you’ll see exactly what “diffusion for text” is doing at every step.
- **Practical**: runs on a single GPU and saves only the final checkpoint by default.
- **Hackable**: a few variables at the top control dataset size, model size, and training budget.

---

## What we’re building (high level)

Autoregressive (AR) LMs generate **left-to-right**:

> token₁ → token₂ → token₃ → …

A masked diffusion LM generates by **iteratively denoising a whole sequence in parallel**:

1. Start from a sequence that is mostly (or entirely) `[MASK]`.
2. Predict tokens for all masked positions.
3. Keep the most confident predictions, re-mask the uncertain ones.
4. Repeat for many steps until no masks remain.

This gives the “cool” effect where the output looks like it’s being *edited into existence*.

---

## Before you run

1. **Runtime → Change runtime type → GPU**
2. (Optional) Set **`RUN_MODE`** below:
   - `"quick"`: small dataset slice, fewer steps (fast to see it work)
   - `"budget_100"`: larger model/dataset/steps (better quality; costs more compute)

> Note: TinyStories teaches the model to write simple stories.  
> If you ask it for Python code, it will still try—but it may answer with story-like text until you train on code data.

In [1]:
# =======================
# 0) Choose a run profile
# =======================

RUN_MODE = "quick"   # "quick" or "budget_100"

# You can always override individual values later.

if RUN_MODE == "quick":
    # Small + fast: good for verifying everything end-to-end
    TRAIN_EXAMPLES = 500
    VAL_EXAMPLES   = 20
    TOKENIZER_TRAIN_EXAMPLES = 300

    SEQ_LEN = 256
    VOCAB_SIZE = 8_000

    D_MODEL = 384
    N_LAYERS = 6
    N_HEADS = 6
    D_FF = 4 * D_MODEL

    DIFFUSION_STEPS = 32

    TRAIN_STEPS = 200
    BATCH_SIZE = 32
    GRAD_ACCUM = 1
    LR = 3e-4
    WEIGHT_DECAY = 0.1
    WARMUP_STEPS = 200

elif RUN_MODE == "budget_100":
    # Heavier: better quality, uses more compute
    TRAIN_EXAMPLES = 1000_000
    VAL_EXAMPLES   = 10_000
    TOKENIZER_TRAIN_EXAMPLES = 150_000

    SEQ_LEN = 256
    VOCAB_SIZE = 26_000

    D_MODEL = 512
    N_LAYERS = 10
    N_HEADS = 8
    D_FF = 4 * D_MODEL

    DIFFUSION_STEPS = 128

    TRAIN_STEPS = 50000
    BATCH_SIZE = 32
    GRAD_ACCUM = 2
    LR = 2e-4
    WEIGHT_DECAY = 0.1
    WARMUP_STEPS = 1_000

else:
    raise ValueError("RUN_MODE must be 'quick' or 'budget_100'")

print("RUN_MODE:", RUN_MODE)

RUN_MODE: quick


# 1) Install dependencies

We’ll use:

- `torch` for training
- `datasets` for TinyStories
- `tokenizers` to train a tokenizer **from scratch**
- `tqdm`, `numpy`, `imageio`, `Pillow` for progress + video export

In [2]:
!pip -q install -U datasets tokenizers accelerate tqdm numpy einops imageio pillow transformers
!pip install hf_transfer

In [3]:
import os, math, time, json, random
import numpy as np
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import IterableDataset, DataLoader

from datasets import load_dataset

print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("bf16 supported:", torch.cuda.is_bf16_supported())

torch: 2.10.0
cuda available: False


In [4]:
!pip uninstall numpy -y --quiet
!pip install numpy==1.23.5 --quiet
train_ds = load_dataset("roneneldan/TinyStories", split=f"train[:{TRAIN_EXAMPLES}]")
val_ds   = load_dataset("roneneldan/TinyStories", split=f"validation[:{VAL_EXAMPLES}]")

print(train_ds, val_ds)
print("\nExample:\n", train_ds[0]["text"][:500])

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/Users/davidbong/Documents/AI_labs/DiffusionModel/Dream7B_LlaDA/.venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
        File "/Users/davidbong/Documents/AI_labs/DiffusionModel/Dream7B_LlaDA/.venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Users/davidbong/Documents/AI_labs/DiffusionModel/Dream7B_LlaDA/.venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 137, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^

# 2) Load TinyStories (small slice)

We’ll load only a **slice** to keep it “not too big”.

# 3) Train a tokenizer from scratch (Byte-level BPE)

We train a Byte-level BPE tokenizer ourselves (no pretrained tokenizer).

Special tokens:

- `[PAD]` padding
- `[UNK]` unknown
- `[BOS]` begin-of-sequence
- `[EOS]` end-of-sequence
- `[MASK]` the diffusion “noise” token
- `<|user|>`, `<|assistant|>`, `<|system|>`, `<|end|>` for chat formatting

In [5]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC
from tokenizers.processors import TemplateProcessing

SPECIAL_TOKENS = [
    "[PAD]", "[UNK]", "[BOS]", "[EOS]", "[MASK]",
    "<|user|>", "<|assistant|>", "<|system|>", "<|end|>",
]

def tokenizer_training_iterator(ds, n_examples):
    for i in range(min(n_examples, len(ds))):
        story = ds[i]["text"].strip()
        yield f"<|user|>\nWrite a short story.\n<|assistant|>\n{story}\n<|end|>\n"

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.normalizer = NFKC()
tokenizer.pre_tokenizer = ByteLevel(add_prefix_space=False)

trainer = BpeTrainer(
    vocab_size=VOCAB_SIZE,
    min_frequency=2,
    special_tokens=SPECIAL_TOKENS,
)

print("Training tokenizer...")
tokenizer.train_from_iterator(
    tokenizer_training_iterator(train_ds, TOKENIZER_TRAIN_EXAMPLES),
    trainer=trainer
)

bos_id = tokenizer.token_to_id("[BOS]")
eos_id = tokenizer.token_to_id("[EOS]")
tokenizer.post_processor = TemplateProcessing(
    single="[BOS] $A [EOS]",
    special_tokens=[("[BOS]", bos_id), ("[EOS]", eos_id)],
)
tokenizer.decoder = ByteLevelDecoder()

TOKENIZER_DIR = "tokenizer_from_scratch"
os.makedirs(TOKENIZER_DIR, exist_ok=True)
TOKENIZER_FILE = os.path.join(TOKENIZER_DIR, "tokenizer.json")
tokenizer.save(TOKENIZER_FILE)

print("Saved tokenizer to:", TOKENIZER_FILE)
print("Vocab size:", tokenizer.get_vocab_size())

Training tokenizer...



Saved tokenizer to: tokenizer_from_scratch/tokenizer.json
Vocab size: 3637


In [6]:
!pip install transformers -U --quiet
from transformers import PreTrainedTokenizerFast

hf_tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_FILE)

hf_tokenizer.pad_token  = "[PAD]"
hf_tokenizer.unk_token  = "[UNK]"
hf_tokenizer.bos_token  = "[BOS]"
hf_tokenizer.eos_token  = "[EOS]"
hf_tokenizer.mask_token = "[MASK]"

hf_tokenizer.add_special_tokens({
    "additional_special_tokens": ["<|user|>", "<|assistant|>", "<|system|>", "<|end|>"]
})

PAD_ID  = hf_tokenizer.pad_token_id
MASK_ID = hf_tokenizer.mask_token_id
BOS_ID  = hf_tokenizer.bos_token_id
EOS_ID  = hf_tokenizer.eos_token_id

print("PAD_ID:", PAD_ID, "MASK_ID:", MASK_ID, "BOS_ID:", BOS_ID, "EOS_ID:", EOS_ID)
print("Example encoding:", hf_tokenizer.encode("Hello world!")[:20])

PAD_ID: 0 MASK_ID: 4 BOS_ID: 2 EOS_ID: 3
Example encoding: [2, 1264, 764, 9, 3]


# 4) Build a tiny diffusion LM (Transformer) from scratch

We implement a minimal **bidirectional Transformer** with:

- token embeddings
- position embeddings
- **time-step embedding** (diffusion step `t`)
- TransformerEncoder blocks
- vocabulary projection head

Training objective: predict original tokens **only at masked positions**.

In [7]:
from dataclasses import dataclass

@dataclass
class DiffusionLMConfig:
    vocab_size: int
    seq_len: int # Context Block
    d_model: int # Head Dimension
    n_layers: int
    n_heads: int
    d_ff: int #Feed Forward 
    dropout: float
    diffusion_steps: int # Diffusion Steps

class DiffusionTransformerLM(nn.Module):
    def __init__(self, cfg: DiffusionLMConfig):
        super().__init__()
        self.cfg = cfg

        self.tok_emb = nn.Embedding(cfg.vocab_size, cfg.d_model)
        self.pos_emb = nn.Embedding(cfg.seq_len, cfg.d_model)
        self.time_emb = nn.Embedding(cfg.diffusion_steps + 1, cfg.d_model)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=cfg.d_model,
            nhead=cfg.n_heads,
            dim_feedforward=cfg.d_ff,
            dropout=cfg.dropout,
            batch_first=True,
            activation="gelu",
            norm_first=True,
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=cfg.n_layers)
        self.ln_f = nn.LayerNorm(cfg.d_model)
        self.lm_head = nn.Linear(cfg.d_model, cfg.vocab_size, bias=False)

        # Tie weights (optional; common in LMs) # Weights Sharing
        self.lm_head.weight = self.tok_emb.weight

        self.drop = nn.Dropout(cfg.dropout)

    def forward(self, input_ids, timesteps, attention_mask=None):
        # input_ids: [B, L]
        # timesteps: [B] integer diffusion step in [1..T]
        # attention_mask: [B, L] bool, True for non-pad tokens

        B, L = input_ids.shape
        if L > self.cfg.seq_len:
            raise ValueError(f"Sequence length {L} > cfg.seq_len {self.cfg.seq_len}")

        pos = torch.arange(L, device=input_ids.device).unsqueeze(0)  # [1, L]
        x = self.tok_emb(input_ids) + self.pos_emb(pos)

        t_emb = self.time_emb(timesteps).unsqueeze(1)  # [B, 1, D]
        x = x + t_emb
        x = self.drop(x)

        if attention_mask is None:
            src_key_padding_mask = None
        else:
            src_key_padding_mask = ~attention_mask  # invert: True = pad/ignore

        x = self.encoder(x, src_key_padding_mask=src_key_padding_mask)
        x = self.ln_f(x)
        logits = self.lm_head(x)  # [B, L, V]
        return logits

cfg = DiffusionLMConfig(
    vocab_size=len(hf_tokenizer),
    seq_len=SEQ_LEN,
    d_model=D_MODEL,
    n_layers=N_LAYERS,
    n_heads=N_HEADS,
    d_ff=D_FF,
    dropout=0.1,
    diffusion_steps=DIFFUSION_STEPS,
)
model = DiffusionTransformerLM(cfg)

n_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {n_params/1e6:.2f}M")

Model parameters: 12.16M


/var/folders/v8/d58dm48534j2765wcq9btnqr0000gn/T/ipykernel_55068/1193024710.py:32: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  self.encoder = nn.TransformerEncoder(enc_layer, num_layers=cfg.n_layers)


# 5) Create a token-block dataset (for language modeling)

We convert each story into a chat-like format:

```
<|user|>
Write a short story.
<|assistant|>
{story}
<|end|>
```

Then we tokenize and stream tokens into fixed-length blocks (`SEQ_LEN`).

In [8]:
def format_as_chat(story_text: str) -> str:
    story_text = story_text.strip()
    return f"<|user|>\nWrite a short story.\n<|assistant|>\n{story_text}\n<|end|>\n"

class TokenBlockDataset(IterableDataset):
    def __init__(self, hf_ds, tokenizer, seq_len, shuffle=False, seed=0):
        self.hf_ds = hf_ds
        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.shuffle = shuffle
        self.seed = seed

    def __iter__(self):
        indices = list(range(len(self.hf_ds)))
        if self.shuffle:
            rng = random.Random(self.seed)
            rng.shuffle(indices)

        buffer = []
        for idx in indices:
            text = format_as_chat(self.hf_ds[idx]["text"])
            ids = self.tokenizer.encode(text, add_special_tokens=True)
            buffer.extend(ids)

            while len(buffer) >= self.seq_len:
                block = buffer[:self.seq_len]
                buffer = buffer[self.seq_len:]
                yield torch.tensor(block, dtype=torch.long)

train_blocks = TokenBlockDataset(train_ds, hf_tokenizer, SEQ_LEN, shuffle=True, seed=42)
val_blocks   = TokenBlockDataset(val_ds,   hf_tokenizer, SEQ_LEN, shuffle=False)

def collate_blocks(batch):
    input_ids = torch.stack(batch, dim=0)  # [B, L]
    attention_mask = (input_ids != PAD_ID)
    return {"input_ids": input_ids, "attention_mask": attention_mask}

train_loader = DataLoader(train_blocks, batch_size=BATCH_SIZE, collate_fn=collate_blocks)
val_loader   = DataLoader(val_blocks,   batch_size=BATCH_SIZE, collate_fn=collate_blocks)

b = next(iter(train_loader))
print({k: v.shape for k, v in b.items()})
print("Decoded snippet:\n", hf_tokenizer.decode(b["input_ids"][0][:120].tolist()))

{'input_ids': torch.Size([32, 256]), 'attention_mask': torch.Size([32, 256])}
Decoded snippet:
 [BOS]<|user|>
Write a short story.
<|assistant|>
Once there was a lady who had a special leg. It was deaf and she never wanted to talk about it.

One day, a little boy asked the lady why her leg was so special. She was surprised he noticed!

The lady smiled and told the boy she was deaf in the leg because she was born that way.

The boy was surprisedâ€” he thought it was unusual but very cool! He asked the lady to teach him sign language and she gladly agreed!

The lady


# 6) Diffusion corruption (masking) + training loss

For each batch:

1. Sample diffusion step `t ∈ {1…T}` for each sequence.
2. Corrupt clean `x₀` into `x_t` by replacing a fraction of tokens with `[MASK]`.
3. Train the model to predict original IDs **only at masked positions**.

In [11]:
def mask_ratio_schedule(t, T: int):
    # Linear schedule: ratio = t/T
    return t.float() / float(T)

@torch.no_grad()
def corrupt_with_mask(input_ids, attention_mask, t, mask_token_id: int, T: int):
    # Returns noisy_ids, labels, mask_positions
    B, L = input_ids.shape
    ratio = mask_ratio_schedule(t, T).unsqueeze(1)  # [B,1]

    can_mask = attention_mask.clone()
    can_mask &= (input_ids != BOS_ID) & (input_ids != EOS_ID) & (input_ids != PAD_ID)

    rand = torch.rand((B, L), device=input_ids.device)
    mask_positions = (rand < ratio) & can_mask

    noisy = input_ids.clone()
    noisy[mask_positions] = mask_token_id

    labels = torch.full_like(input_ids, -100)
    labels[mask_positions] = input_ids[mask_positions]

    return noisy, labels, mask_positions

def diffusion_loss(model, batch, T: int):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]

    B = input_ids.size(0)
    t = torch.randint(1, T + 1, (B,), device=input_ids.device)

    noisy_ids, labels, _ = corrupt_with_mask(
        input_ids=input_ids,
        attention_mask=attention_mask,
        t=t,
        mask_token_id=MASK_ID,
        T=T,
    )

    logits = model(noisy_ids, timesteps=t, attention_mask=attention_mask)  # [B,L,V]
    loss = F.cross_entropy(
        logits.view(-1, logits.size(-1)),
        labels.view(-1),
        ignore_index=-100,
    )
    return loss

# 7) Train (from scratch)

By default this notebook saves **only the final checkpoint**:

- `checkpoints/final/model.pt`
- `checkpoints/final/config.json`
- `checkpoints/final/tokenizer/`

In [12]:
from accelerate import Accelerator
from transformers import get_cosine_schedule_with_warmup

accelerator = Accelerator(mixed_precision="bf16" if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else "fp16")
device = accelerator.device

model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=TRAIN_STEPS,
)

model, optimizer, train_loader, val_loader, scheduler = accelerator.prepare(
    model, optimizer, train_loader, val_loader, scheduler
)

def eval_loss(n_batches=20):
    model.eval()
    losses = []
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            if i >= n_batches:
                break

            loss = diffusion_loss(model, batch, T=cfg.diffusion_steps)

            # gather across processes -> always make it 1D
            gathered = accelerator.gather(loss.detach().float().reshape(1))

            # now gathered is shape [world_size] (or [1] on single GPU)
            losses.append(gathered.cpu())

    model.train()

    if len(losses) == 0:
        return float("nan")

    losses = torch.cat(losses)   # safe: all are 1D tensors
    return losses.mean().item()


model.train()
pbar = tqdm(range(TRAIN_STEPS), disable=not accelerator.is_main_process)
running = []

train_iter = iter(train_loader)

for step in pbar:
    try:
        batch = next(train_iter)
    except StopIteration:
        train_iter = iter(train_loader)
        batch = next(train_iter)

    loss = diffusion_loss(model, batch, T=cfg.diffusion_steps) / GRAD_ACCUM
    accelerator.backward(loss)

    if (step + 1) % GRAD_ACCUM == 0:
        accelerator.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    running.append(loss.item() * GRAD_ACCUM)

    if (step + 1) % 50 == 0 and accelerator.is_main_process:
        pbar.set_description(f"loss={np.mean(running[-50:]):.4f} lr={scheduler.get_last_lr()[0]:.2e}")

    if (step + 1) % 500 == 0 and accelerator.is_main_process:
        val_l = eval_loss(n_batches=10)
        print(f"\nStep {step+1} | val_loss ~ {val_l:.4f}")

if accelerator.is_main_process:
    OUT_DIR = "checkpoints/final"
    os.makedirs(OUT_DIR, exist_ok=True)
    torch.save(accelerator.unwrap_model(model).state_dict(), os.path.join(OUT_DIR, "model.pt"))
    with open(os.path.join(OUT_DIR, "config.json"), "w") as f:
        json.dump(cfg.__dict__, f, indent=2)
    hf_tokenizer.save_pretrained(os.path.join(OUT_DIR, "tokenizer"))
    print("Saved final checkpoint to:", OUT_DIR)

  0%|          | 0/200 [00:00<?, ?it/s]

Saved final checkpoint to: checkpoints/final


# 8) Diffusion sampling (progressive unmasking)

Prompt tokens are **fixed**. The “assistant answer” region starts fully masked.

At each step we:
- predict tokens for masked positions
- keep the most confident tokens
- re-mask the least confident tokens

We record intermediate steps for a GIF.

In [ ]:
@torch.no_grad()
def diffusion_generate(
    model,
    tokenizer,
    prompt_text: str,
    max_new_tokens: int = 128,
    diffusion_steps: int = 64,
    temperature: float = 1.0,
    top_k: int = 0,
    record_steps: bool = True,
):
    model.eval()
    device = next(model.parameters()).device

    prompt_ids = tokenizer.encode(prompt_text, add_special_tokens=True)
    prompt_ids = torch.tensor(prompt_ids, dtype=torch.long, device=device).unsqueeze(0)  # [1, Lp]

    Lp = prompt_ids.size(1)
    L = min(cfg.seq_len, Lp + max_new_tokens)
    gen_len = L - Lp

    x = torch.full((1, L), MASK_ID, dtype=torch.long, device=device)
    x[:, :Lp] = prompt_ids[:, :Lp]

    fixed = torch.zeros((1, L), dtype=torch.bool, device=device)
    fixed[:, :Lp] = True

    attention_mask = torch.ones((1, L), dtype=torch.bool, device=device)

    frames = []
    
    def sample_from_logits(logits):
        if temperature != 1.0:
            logits = logits / temperature

        if top_k and top_k > 0:
            topk_vals, topk_idx = torch.topk(logits, k=top_k, dim=-1)
            filtered = torch.full_like(logits, float("-inf"))
            filtered.scatter_(-1, topk_idx, topk_vals)
            logits = filtered

        probs = F.softmax(logits, dim=-1)
        flat = probs.view(-1, probs.size(-1))
        sampled = torch.multinomial(flat, num_samples=1).view(1, L)
        sampled_prob = probs.gather(-1, sampled.unsqueeze(-1)).squeeze(-1)  # [1,L]
        return sampled, sampled_prob

    for s in range(diffusion_steps, 0, -1):
        t = torch.tensor([s], device=device, dtype=torch.long)
        logits = model(x, timesteps=t, attention_mask=attention_mask)
        sampled, conf = sample_from_logits(logits)

        update_pos = ~fixed
        x[update_pos] = sampled[update_pos]

        next_ratio = float(s - 1) / float(diffusion_steps)
        target_masks = int(math.ceil(gen_len * next_ratio))

        gen_positions = torch.arange(L, device=device) >= Lp
        candidates = gen_positions & (~fixed[0])
        cand_idx = torch.where(candidates)[0]

        if target_masks > 0 and cand_idx.numel() > 0:
            cand_conf = conf[0, cand_idx]
            k = min(target_masks, cand_idx.numel())
            _, low_idx = torch.topk(cand_conf, k=k, largest=False)
            remask_positions = cand_idx[low_idx]
            x[0, remask_positions] = MASK_ID

        if record_steps:
            decoded = tokenizer.decode(x[0].tolist())
            decoded = decoded.replace("[MASK]", "█")
            frames.append((s, decoded))

    final = tokenizer.decode(x[0].tolist())
    model.train()
    return final, frames

def chat_prompt(user_msg: str, system_msg: str = None) -> str:
    parts = []
    if system_msg:
        parts.append(f"<|system|>\n{system_msg}\n")
    parts.append(f"<|user|>\n{user_msg}\n")
    parts.append("<|assistant|>\n")
    return "".join(parts)

TEST_USER_PROMPT = "Once upon a time"
prompt_text = chat_prompt(TEST_USER_PROMPT)

final_text, frames = diffusion_generate(
    model=accelerator.unwrap_model(model),
    tokenizer=hf_tokenizer,
    prompt_text=prompt_text,
    max_new_tokens=128,
    diffusion_steps=cfg.diffusion_steps,
    temperature=1.0,
    top_k=50,
    record_steps=True,
)

print("Final decoded (raw):\n")
print(final_text[:1000])
print("\nRecorded frames:", len(frames))